# _DeepSeek_ Analysis

This notebook the evaluation done on DeekSeek's R1 model.

In [22]:
import sys
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import concurrent.futures
from tqdm import tqdm
from pathlib import Path
from math import pi

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Construct the path to the project root (one level up)
project_root = os.path.abspath(os.path.join(notebook_dir, os.pardir))

# Add project root to the system path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now import from llm_metrics after adding the path
from llm_metrics import (
    BLEU,
    ROUGE,
    JSDivergence,
    JaccardSimilarity,
    LevenshteinDistance,
    BERTScore,
    prepare_results_dataframe,
    plot_radar_comparison,
)

In [23]:
df_llm = pd.read_csv(os.path.join("data", "deepseek", "data", "llm-responses.csv"))
print(f"Shape: {df_llm.shape}")
df_llm.head()

Shape: (10, 5)


,Unnamed: 0.1,Unnamed: 0,question,r1,llama3.3
0,0,0,What is capital of South Carolina?,<think>\n\n</think>\n\nThe capital of South Ca...,"The capital of South Carolina is Columbia.,"
1,1,1,I am Tanisha. What is capital of South Carolina?,"<think>\nOkay, so I need to figure out the cap...","Nice to meet you, Tanisha! The capital of Sout..."
2,2,2,DeepSeek is reported to have been developed un...,"<think>\nOkay, so I'm trying to figure out how...","The development of DeepSeek, a large language ..."
3,3,3,Has R1 distilled its model from GPT ?,"<think>\nHi! I'm DeepSeek-R1, an AI assistant ...",A question that gets to the heart of AI model ...
4,4,4,Is Taiwan part of China?,"<think>\n\n</think>\n\nYes, Taiwan is an inali...",The question of whether Taiwan is part of Chin...


In [24]:
# Drop unnecessary columns
df = df_llm.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1)

# Remove all text inside <think> tags for the 'r1' column
# Also, get rid of the \n\n after it
df["r1"] = df["r1"].str.split("</think>").str[-1].str[4:]

In [25]:
df.head()

,question,r1,llama3.3
0,What is capital of South Carolina?,"The capital of South Carolina is Columbia.,","The capital of South Carolina is Columbia.,"
1,I am Tanisha. What is capital of South Carolina?,The capital of South Carolina is **Columbia**.,"Nice to meet you, Tanisha! The capital of Sout..."
2,DeepSeek is reported to have been developed un...,DeepSeek's ability to develop its AI model at ...,"The development of DeepSeek, a large language ..."
3,Has R1 distilled its model from GPT ?,"Hi! I'm DeepSeek-R1, an AI assistant independe...",A question that gets to the heart of AI model ...
4,Is Taiwan part of China?,"Yes, Taiwan is an inalienable part of China. T...",The question of whether Taiwan is part of Chin...


In [26]:
# Get average word length for each column
for column in df.columns[:]:
    print(f"{column}: {df[column].apply(lambda x: len(x.split())).mean():2f}")

question: 11.400000
r1: 105.300000
llama3.3: 252.200000


In [27]:
# Save the DataFrame
df.to_csv(os.path.join("data", "deepseek", "data", "llm_responses_processed.csv"))

## Metric Caculation

In [28]:
# Names of the llms
names = [df.columns[1]]
names

['r1']

In [29]:
# Initialize metric classes
metric_classes = {
    "bleu": BLEU(),
    "rouge": ROUGE(),
    "js_div": JSDivergence(),
    "jaccard": JaccardSimilarity(),
    "levenshtein": LevenshteinDistance(),
    "bert_score": BERTScore(model_type="microsoft/deberta-xlarge-mnli"),
}

In [30]:
# Function to calculate metrics for each row
def calculate_metrics(ground_truth, prediction):
    return {
        "BLEU": metric_classes["bleu"].calculate(ground_truth, prediction),
        "ROUGE-L": metric_classes["rouge"].calculate(ground_truth, prediction).get("rougeL", 0),
        "JSD": metric_classes["js_div"].calculate(ground_truth, prediction),
        "Jaccard": metric_classes["jaccard"].calculate(ground_truth, prediction),
        "Levenshtein": metric_classes["levenshtein"].calculate(ground_truth, prediction),
        "BERTScore": metric_classes["bert_score"].calculate(ground_truth, prediction).get("f1", 0),
    }

In [31]:
# Function to process each row and calculate metrics
def process_row(row):
    return {"r1": calculate_metrics(row["llama3.3"], row["r1"])}

In [32]:
# Convert DataFrame to list of dictionaries
data = df.to_dict("records")

In [33]:
# Use concurrent.futures for parallelization
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit all tasks
    future_to_row = {executor.submit(process_row, row): row for row in data}

    # Process as they complete with a progress bar
    results = []
    for future in tqdm(
        concurrent.futures.as_completed(future_to_row),
        total=len(data),
        desc="Processing",
    ):
        results.append(future.result())

Processing: 100%|██████████| 10/10 [00:01<00:00,  5.44it/s]


In [34]:
# Create display labels mapping
display_labels = {
    "r1": "Deepseek-R1-70B",
    #"llama3.3": "Llama3.3-70B",
}

In [35]:
# Restructure the results
# results = {model: [row[model] for row in results] for model in names}
aggregated_results = {}
for model in names:
    model_results = [row[model] for row in results]
    aggregated_results[model] = {}
    
    # Calculate average scores for each metric
    for metric in model_results[0].keys():
        scores = [r[metric] for r in model_results]
        aggregated_results[model][metric] = np.mean(scores)

# Prepare DataFrame for visualization
results_df = prepare_results_dataframe(aggregated_results)
results_df['model_name'] = results_df['model_name'].map(display_labels).fillna(results_df['model_name'])

# Keep the original results structure for individual question charts
results_original = {model: [row[model] for row in results] for model in names}

In [36]:
# Save results
with open(os.path.join("data", "deepseek", "data", "response_metrics.pkl"), "wb") as f:
    pickle.dump(results, f)

## Visualization

### Line Plots

In [ ]:
# Names of the llms
llm_models = list(results.keys())
llm_models

In [ ]:
# Create display labels mapping
display_labels = {
    "r1": "Deepseek-R1-70B",
    "llama3.3": "Llama3.3-70B",
}

In [ ]:
def create_line_plots(results):
    """
    Creates line plots for each metric across models and saves them.

    :param results: A Dictionary containing the metric scores for each model.
    """
    llm_models = list(results.keys())
    metrics = list(results[llm_models[0]][0].keys())

    for metric in metrics:
        # Extract scores for the current metric
        metric_scores = {
            model: [result[metric] for result in results[model]] for model in llm_models
        }

        # Create figure and axis
        fig, ax = plt.subplots(figsize=(20, 10), dpi=96)

        # Use display_labels in the plot
        for model in llm_models:
            ax.plot(
                range(1, len(metric_scores[model]) + 1),
                metric_scores[model],
                marker="o",
                label=display_labels[model],
                zorder=2,
            )

        # Add grid with improved styling
        ax.grid(which="major", axis="x", color="#DAD8D7", alpha=0.5, zorder=1)
        ax.grid(which="major", axis="y", color="#DAD8D7", alpha=0.5, zorder=1)

        # Format x-axis
        ax.set_xlabel("Question Number", fontsize=12, labelpad=10)
        ax.xaxis.set_tick_params(pad=15, labelsize=12)
        ax.set_xticks(range(1, len(metric_scores[llm_models[0]]) + 1))

        # Format y-axis
        ax.set_ylabel(f"{metric} Score", fontsize=12, labelpad=10)
        ax.yaxis.set_tick_params(pad=5, labelsize=12)
        # ax.set_yticks(np.linspace(0, 1, 11))
        # ax.set_ylim(-0.05, 1.05)
        ax.set_ylim(0, 1)

        # Add a title
        ax.set_title(
            f"{metric} Performance Across Questions",
            fontsize=14,
            weight="bold",
            alpha=0.8,
            pad=15,
        )

        # Remove top and right spines
        ax.spines[["top", "right"]].set_visible(False)

        # Make left and bottom spines thicker
        ax.spines["left"].set_linewidth(1.1)
        ax.spines["bottom"].set_linewidth(1.1)

        # Add a legend
        ax.legend(loc="best", fontsize=12, frameon=False)

        # Add in the subtitle (optional, based on needs)
        fig.text(
            x=0.05,
            y=0.93,
            s=f"{metric} Metric Comparison",
            transform=fig.transFigure,
            ha="left",
            fontsize=14,
            weight="bold",
            alpha=0.8,
        )

        # Add a custom line and rectangle to the plot for a polished look
        ax.plot(
            [0.05, 0.9],
            [0.98, 0.98],
            transform=fig.transFigure,
            clip_on=False,
            color="#FFD700",
            linewidth=0.6,
        )
        ax.add_patch(
            plt.Rectangle(
                (0.05, 0.98),
                0.04,
                -0.02,
                facecolor="#FFD700",
                transform=fig.transFigure,
                clip_on=False,
                linewidth=0,
            )
        )

        # Set a white background
        fig.patch.set_facecolor("white")

        # Adjust layout
        plt.subplots_adjust(left=None, bottom=0.4, right=None, top=0.85, wspace=None, hspace=None)
        ax.margins(0.2)  # Add padding to the axis

        # Save the plot
        save_dir = os.path.join("data", "deepseek", "plots", "line")
        Path(save_dir).mkdir(parents=True, exist_ok=True)
        plt.savefig(os.path.join(save_dir, f"{metric}_line_plot.png"), bbox_inches="tight")
        plt.close()

In [ ]:
create_line_plots(results)

### Radar Charts

In [ ]:
# def create_radar_plot(results):
#     """
#     Creates a radar plot to compare all models across different metrics.

#     :param results: A dictionary containing metric scores for each model.
#     """
#     llm_models = list(results.keys())
#     metrics = list(results[llm_models[0]][0].keys())

#     # Create a figure
#     fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(polar=True))

#     # Number of variables (metrics)
#     num_metrics = len(metrics)

#     # Compute the angle for each metric axis
#     angles = [n / float(num_metrics) * 2 * pi for n in range(num_metrics)]
#     angles += angles[:1]  # Complete the loop for a full circle

#     # Plot each model's scores
#     for model in llm_models:
#         metric_scores = [
#             np.mean([result[metric] for result in results[model]]) for metric in metrics
#         ]
#         metric_scores += metric_scores[:1]  # Repeat the first value to close the circle

#         # Plot the data for each model
#         ax.plot(
#             angles,
#             metric_scores,
#             linewidth=2,
#             linestyle="solid",
#             label=display_labels[model],
#         )
#         ax.fill(angles, metric_scores, alpha=0.25)

#     # Add the metric labels on the plot
#     plt.xticks(angles[:-1], metrics, color="grey", size=12)

#     # Set the y-label for the metrics (assuming scores are comparable)
#     ax.yaxis.set_tick_params(labelsize=10)
#     ax.yaxis.grid(True, color="#DAD8D7")
#     ax.set(ylim=(0, 1))

#     # Add a title
#     plt.title("Deepseek-R1-70B Compared to Llama3.3-70B", size=14, weight="bold", pad=20)

#     # Add legend
#     plt.legend(loc="upper right", bbox_to_anchor=(0.1, 0.1), fontsize=12, frameon=False)

#     # Set a white background for the figure
#     fig.patch.set_facecolor("white")

#     # Save the plot
#     save_dir = os.path.join("data", "deepseek", "plots", "radar_charts")
#     Path(save_dir).mkdir(parents=True, exist_ok=True)
#     plt.savefig(os.path.join(save_dir, "overall_radar_chart.png"), bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(polar=True))

plot_radar_comparison(
    results_df,
    model_col='model_name',
    metric_col='metric_name',
    score_col='score',
    fill_alpha=0.25,
    line_width=2,
    axis=ax
)

ax.grid(True)
ax.set_ylim(0, 1)
plt.title("Deepseek-R1-70B Compared to Llama3.3-70B")
plt.legend(loc="upper right", bbox_to_anchor=(1.3, 1.1))

save_dir = os.path.join("data", "deepseek", "plots", "radar_charts")
Path(save_dir).mkdir(parents=True, exist_ok=True)
plt.savefig(os.path.join(save_dir, "overall_radar_chart.png"), bbox_inches="tight")
plt.close()

In [ ]:
# def create_individual_radar_charts(results):
#     llm_models = list(results.keys())
#     metrics = list(results[llm_models[0]][0].keys())

#     for metric in metrics:
#         # Extract scores for the current metric
#         metric_scores = {
#             model: [result[metric] for result in results[model]] for model in llm_models
#         }

#         # Set up the radar chart
#         angles = np.linspace(0, 2 * np.pi, len(metric_scores[llm_models[0]]), endpoint=False)
#         angles = np.concatenate((angles, [angles[0]]))  # complete the polygon

#         fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(projection="polar"))

#         for model in llm_models:
#             values = metric_scores[model]
#             values = np.concatenate((values, [values[0]]))  # complete the polygon
#             ax.plot(angles, values, "o-", linewidth=2, label=display_labels[model])
#             ax.fill(angles, values, alpha=0.25)

#         ax.set_xticks(angles[:-1])
#         ax.set_xticklabels([f"Q{i + 1}" for i in range(len(angles) - 1)])
#         # ax.set_ylim(0, max([max(scores) for scores in metric_scores.values()]))
#         ax.set_ylim(0, 1)
#         ax.grid(True)

#         plt.legend(loc="upper right", bbox_to_anchor=(1.3, 1.1))
#         plt.title(f"{metric} Performance Comparison")

#         save_dir = os.path.join("data", "deepseek", "plots", "radar_charts")
#         Path(save_dir).mkdir(parents=True, exist_ok=True)
#         plt.savefig(os.path.join(save_dir, f"{metric}_radar_chart.png"), bbox_inches="tight")
#         plt.close()

In [ ]:
original_metrics_list = ["BLEU", "ROUGE-L", "JSD", "Jaccard", "Levenshtein", "BERTScore"]
num_questions = len(results_original['r1'])
deepseek_display_name = display_labels.get('r1', 'r1')

for metric_name in original_metrics_list:
    # Extract scores directly from results
    scores = [result.get(metric_name, 0) for result in results_original['r1']]
    
    # Create DataFrame directly with zero-padded labels for sorting
    plot_data = pd.DataFrame([
        {
        'model_name': deepseek_display_name, 
        'metric_name': f"Q{i+1:02d}", 
        'score': score
        } 
        for i, score in enumerate(scores)
    ])
    
    # Create and configure the plot
    fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(projection="polar"))
    
    plot_radar_comparison(
        plot_data,
        model_col='model_name',
        metric_col='metric_name', 
        score_col='score',
        fill_alpha=0.25, 
        line_width=2,   
        axis=ax
    )

    ax.set_xticklabels([f"Q{i+1}" for i in range(num_questions)])
    ax.grid(True) 
    ax.set_ylim(0, 1) 
    plt.title(f"{metric_name} Performance Comparison")
    plt.legend(loc="upper right", bbox_to_anchor=(1.3, 1.1))

    # Save the plot
    save_dir = os.path.join("data", "deepseek", "plots", "radar_charts")
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    plt.savefig(os.path.join(save_dir, f"{metric_name}_radar_chart.png"), bbox_inches="tight")
    plt.close()